In [1]:
import sqlalchemy
sqlalchemy.__version__

'1.3.3'

In [5]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///memory:', echo=True)

In [11]:
from sqlalchemy.ext.declarative import declarative_base
Base= declarative_base()

In [12]:
from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__='users'
    
    id=Column(Integer, primary_key=True)
    name= Column(String)
    fullname=Column(String)
    nickname=Column(String)
    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (self.name, self.fullname, self.nickname)

In [13]:
User.__table__


Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [14]:
Base.metadata.create_all(engine)

2019-10-26 15:20:41,867 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-10-26 15:20:41,868 INFO sqlalchemy.engine.base.Engine ()
2019-10-26 15:20:41,870 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-10-26 15:20:41,871 INFO sqlalchemy.engine.base.Engine ()
2019-10-26 15:20:41,873 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-10-26 15:20:41,875 INFO sqlalchemy.engine.base.Engine ()
2019-10-26 15:20:41,879 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2019-10-26 15:20:41,880 INFO sqlalchemy.engine.base.Engine ()
2019-10-26 15:20:41,883 INFO sqlalchemy.engine.base.Engine COMMIT


In [15]:
ed_user= User(name='ed', fullname='Ed Jones', nickname='edsnickname')
ed_user.name

'ed'

In [16]:
ed_user.nickname

'edsnickname'

In [17]:
str(ed_user.id)

'None'

In [18]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [19]:
session = Session()

In [20]:
ed_user= User(name='ed', fullname='Ed Jones', nickname='edsnickname')

In [22]:
session.add(ed_user)

In [23]:
our_user = session.query(User).filter_by(name='ed').first()

2019-10-26 15:53:50,788 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-10-26 15:53:50,791 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-10-26 15:53:50,792 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edsnickname')
2019-10-26 15:53:50,797 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2019-10-26 15:53:50,798 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


In [24]:
our_user


<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [25]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', nickname='windy'),
    User(name='mary', fullname='Mary Contrary', nickname='mary'),
    User(name='fred', fullname='Fred Flintstone', nickname='freddy')
])

In [26]:
ed_user.nickname='eddie'

In [29]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', nickname='eddie')>])

In [30]:
session.new

IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <User(name='mary', fullname='Mary Contrary', nickname='mary')>, <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>])

In [31]:
session.commit()

2019-10-26 16:00:35,305 INFO sqlalchemy.engine.base.Engine UPDATE users SET nickname=? WHERE users.id = ?
2019-10-26 16:00:35,307 INFO sqlalchemy.engine.base.Engine ('eddie', 1)
2019-10-26 16:00:35,308 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-10-26 16:00:35,310 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'windy')
2019-10-26 16:00:35,313 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-10-26 16:00:35,314 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'mary')
2019-10-26 16:00:35,316 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-10-26 16:00:35,318 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flintstone', 'freddy')
2019-10-26 16:00:35,319 INFO sqlalchemy.engine.base.Engine COMMIT


In [32]:
ed_user.id

2019-10-26 16:01:15,861 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-10-26 16:01:15,863 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2019-10-26 16:01:15,864 INFO sqlalchemy.engine.base.Engine (1,)


1

In [33]:
ed_user.name = 'Edwardo'

In [38]:
fake_user =User(name='fakeuser', fullname='Invalid', nickname='123456')
session.add(fake_user)

In [39]:
session.query(User).filter(User.name.in_(['Edwardo','fakeuser'])).all()

2019-10-26 16:07:57,278 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-10-26 16:07:57,279 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '123456')
2019-10-26 16:07:57,281 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2019-10-26 16:07:57,282 INFO sqlalchemy.engine.base.Engine ('Edwardo', 'fakeuser')


[<User(name='Edwardo', fullname='Ed Jones', nickname='eddie')>,
 <User(name='fakeuser', fullname='Invalid', nickname='123456')>]

In [40]:
session.rollback()

2019-10-26 16:08:18,842 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [41]:
ed_user.name

2019-10-26 16:08:36,475 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-10-26 16:08:36,477 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2019-10-26 16:08:36,479 INFO sqlalchemy.engine.base.Engine (1,)


'ed'

In [42]:
fake_user in session

False

In [43]:
session.query(User).filter(User.name.in_(['ed','fakeuser'])).all()

2019-10-26 16:10:26,924 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2019-10-26 16:10:26,926 INFO sqlalchemy.engine.base.Engine ('ed', 'fakeuser')


[<User(name='ed', fullname='Ed Jones', nickname='eddie')>]

In [44]:
for instance in session.query(User).order_by(User.id):
    print(instance.name,instance.fullname)

2019-10-26 16:28:21,692 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
2019-10-26 16:28:21,693 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [46]:
for name,fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2019-10-26 16:29:47,933 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2019-10-26 16:29:47,935 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [47]:
for row in session.query(User, User.name).all():
    print(row.User, row.name)

2019-10-26 16:31:33,759 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users
2019-10-26 16:31:33,761 INFO sqlalchemy.engine.base.Engine ()
<User(name='ed', fullname='Ed Jones', nickname='eddie')> ed
<User(name='wendy', fullname='Wendy Williams', nickname='windy')> wendy
<User(name='mary', fullname='Mary Contrary', nickname='mary')> mary
<User(name='fred', fullname='Fred Flintstone', nickname='freddy')> fred


In [48]:
for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

2019-10-26 16:33:19,651 INFO sqlalchemy.engine.base.Engine SELECT users.name AS name_label 
FROM users
2019-10-26 16:33:19,652 INFO sqlalchemy.engine.base.Engine ()
ed
wendy
mary
fred


In [49]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

2019-10-26 16:35:32,231 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2019-10-26 16:35:32,233 INFO sqlalchemy.engine.base.Engine (2, 1)
<User(name='wendy', fullname='Wendy Williams', nickname='windy')>
<User(name='mary', fullname='Mary Contrary', nickname='mary')>


In [50]:
for name, in session.query(User.name).filter(User.fullname=='Ed Jones'):
    print(name)

2019-10-26 16:37:03,389 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2019-10-26 16:37:03,391 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)
ed


In [51]:
for user in session.query(User).filter(User.name=='ed').filter(User.fullname=='Ed Jones'):
    print(user)

2019-10-26 16:38:11,093 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2019-10-26 16:38:11,094 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones')
<User(name='ed', fullname='Ed Jones', nickname='eddie')>


In [53]:
for user in session.query(User).filter(User.name.in_(['ed', 'wendy','jack'])):
    print(user)

2019-10-26 16:41:37,273 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?, ?)
2019-10-26 16:41:37,274 INFO sqlalchemy.engine.base.Engine ('ed', 'wendy', 'jack')
<User(name='ed', fullname='Ed Jones', nickname='eddie')>
<User(name='wendy', fullname='Wendy Williams', nickname='windy')>


In [54]:
query =session.query(User).filter(User.name.like('%ed')).order_by(User.id)

In [55]:
query.all()

2019-10-26 16:43:41,982 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2019-10-26 16:43:41,984 INFO sqlalchemy.engine.base.Engine ('%ed',)


[<User(name='ed', fullname='Ed Jones', nickname='eddie')>,
 <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>]

In [56]:
query.first()

2019-10-26 16:58:00,924 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
 LIMIT ? OFFSET ?
2019-10-26 16:58:00,925 INFO sqlalchemy.engine.base.Engine ('%ed', 1, 0)


<User(name='ed', fullname='Ed Jones', nickname='eddie')>

In [57]:
user = query.one()

2019-10-26 16:58:19,158 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2019-10-26 16:58:19,160 INFO sqlalchemy.engine.base.Engine ('%ed',)


MultipleResultsFound: Multiple rows were found for one()

In [58]:
user = query.filter(User.id == 99).one()

2019-10-26 16:59:27,570 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? AND users.id = ? ORDER BY users.id
2019-10-26 16:59:27,572 INFO sqlalchemy.engine.base.Engine ('%ed', 99)


NoResultFound: No row was found for one()

In [59]:
query =session.query(User.id).filter(User.name == 'ed').order_by(User.id)

In [61]:
query.scalar()

2019-10-26 17:05:58,993 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id 
FROM users 
WHERE users.name = ? ORDER BY users.id
2019-10-26 17:05:58,996 INFO sqlalchemy.engine.base.Engine ('ed',)


1

In [62]:
from sqlalchemy import text
for user in session.query(User).filter(text('id<244')).order_by(text('id')).all():
    print(user.name)

2019-10-26 17:08:29,830 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE id<244 ORDER BY id
2019-10-26 17:08:29,832 INFO sqlalchemy.engine.base.Engine ()
ed
wendy
mary
fred


In [63]:
for user in session.query(User).filter(text('id>244')).order_by(text('id')).all():
    print(user.name)

2019-10-26 17:08:57,798 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE id>244 ORDER BY id
2019-10-26 17:08:57,800 INFO sqlalchemy.engine.base.Engine ()


In [64]:
session.query(User).filter(text("id<:value and name=:name")).params(value=224,name='fred').order_by(User.id).one()

2019-10-26 17:11:45,285 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE id<? and name=? ORDER BY users.id
2019-10-26 17:11:45,286 INFO sqlalchemy.engine.base.Engine (224, 'fred')


<User(name='fred', fullname='Fred Flintstone', nickname='freddy')>

In [65]:
session.query(User).from_statement(
                                   text("SELECT * FROM users where name=:name")).params(name='ed').all()

2019-10-26 17:14:00,660 INFO sqlalchemy.engine.base.Engine SELECT * FROM users where name=?
2019-10-26 17:14:00,662 INFO sqlalchemy.engine.base.Engine ('ed',)


[<User(name='ed', fullname='Ed Jones', nickname='eddie')>]

In [67]:
stmt  = text("SELECT name, id, fullname, nickname FROM users where name=:name")
stmt = stmt.columns(User.name, User.id, User.fullname, User.nickname)
session.query(User).from_statement(stmt).params(name='ed').all()

2019-10-26 17:20:38,835 INFO sqlalchemy.engine.base.Engine SELECT name, id, fullname, nickname FROM users where name=?
2019-10-26 17:20:38,837 INFO sqlalchemy.engine.base.Engine ('ed',)


[<User(name='ed', fullname='Ed Jones', nickname='eddie')>]

In [69]:
stmt = text("SELECT name, id FROM users where name=:name")
stmt = stmt.columns(User.name, User.id)
session.query(User.id, User.name).from_statement(stmt).params(name='ed').all()

2019-10-26 17:30:33,393 INFO sqlalchemy.engine.base.Engine SELECT name, id FROM users where name=?
2019-10-26 17:30:33,394 INFO sqlalchemy.engine.base.Engine ('ed',)


[(1, 'ed')]

In [70]:
session.query(User).filter(User.name.like("%ed")).count()

2019-10-26 17:31:43,948 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ?) AS anon_1
2019-10-26 17:31:43,949 INFO sqlalchemy.engine.base.Engine ('%ed',)


2

In [72]:
from sqlalchemy import func
session.query(func.count(User.name),User.name).group_by(User.name).all()

2019-10-26 17:33:53,196 INFO sqlalchemy.engine.base.Engine SELECT count(users.name) AS count_1, users.name AS users_name 
FROM users GROUP BY users.name
2019-10-26 17:33:53,197 INFO sqlalchemy.engine.base.Engine ()


[(1, 'ed'), (1, 'fred'), (1, 'mary'), (1, 'wendy')]

In [73]:
session.query(func.count('*')).select_from(User).scalar()

2019-10-26 17:41:54,372 INFO sqlalchemy.engine.base.Engine SELECT count(?) AS count_1 
FROM users
2019-10-26 17:41:54,374 INFO sqlalchemy.engine.base.Engine ('*',)


4

In [74]:
session.query(func.count(User.id)).scalar()

2019-10-26 17:42:37,247 INFO sqlalchemy.engine.base.Engine SELECT count(users.id) AS count_1 
FROM users
2019-10-26 17:42:37,249 INFO sqlalchemy.engine.base.Engine ()


4

In [76]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship


class Address(Base):
    __tablename__='addresses'
    id = Column(Integer, primary_key=True)
    email_address=Column(String, nullable=False)
    user_id= Column(Integer, ForeignKey('users.id'))
    user=relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address

In [77]:
User.addresses = relationship("Address", order_by="Address.id", back_populates="user")

In [78]:
Base.metadata.create_all(engine)

2019-10-26 18:01:46,217 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-10-26 18:01:46,219 INFO sqlalchemy.engine.base.Engine ()
2019-10-26 18:01:46,221 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-10-26 18:01:46,223 INFO sqlalchemy.engine.base.Engine ()
2019-10-26 18:01:46,226 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2019-10-26 18:01:46,227 INFO sqlalchemy.engine.base.Engine ()
2019-10-26 18:01:46,231 INFO sqlalchemy.engine.base.Engine COMMIT


In [79]:
jack =User(name='jack', fullname='Jack Bean', nickname='gjffdd')
jack.addresses


[]

In [83]:
jack.addresses= [
    Address(email_address='jack@google.com'),
    Address(email_address='j25@google.com')
]

In [84]:
jack.addresses[1]

<Address(email_address='j25@google.com')>

In [86]:
jack.addresses[1].user

<User(name='jack', fullname='Jack Bean', nickname='gjffdd')>

In [87]:
session.add(jack)
session.commit()

2019-10-26 21:11:36,196 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-10-26 21:11:36,198 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Bean', 'gjffdd')
2019-10-26 21:11:36,203 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2019-10-26 21:11:36,205 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 5)
2019-10-26 21:11:36,208 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2019-10-26 21:11:36,209 INFO sqlalchemy.engine.base.Engine ('j25@google.com', 5)
2019-10-26 21:11:36,211 INFO sqlalchemy.engine.base.Engine COMMIT


In [88]:
jack=session.query(User).filter_by(name='jack').one()

2019-10-26 21:13:18,052 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-10-26 21:13:18,054 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-10-26 21:13:18,056 INFO sqlalchemy.engine.base.Engine ('jack',)


In [89]:
jack

<User(name='jack', fullname='Jack Bean', nickname='gjffdd')>

In [90]:
jack.addresses

2019-10-26 21:13:53,914 INFO sqlalchemy.engine.base.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE ? = addresses.user_id ORDER BY addresses.id
2019-10-26 21:13:53,915 INFO sqlalchemy.engine.base.Engine (5,)


[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@google.com')>]

In [91]:
for u,a in session.query(User, Address).filter(User.id==Address.user_id).filter(Address.email_address=='jack@google.com').all():
    print(u)
    print(a)

2019-10-26 21:16:07,939 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM users, addresses 
WHERE users.id = addresses.user_id AND addresses.email_address = ?
2019-10-26 21:16:07,941 INFO sqlalchemy.engine.base.Engine ('jack@google.com',)
<User(name='jack', fullname='Jack Bean', nickname='gjffdd')>
<Address(email_address='jack@google.com')>


In [92]:
session.query(User).join(Address).filter(Address.email_address=='jack@google.com').all()

2019-10-26 21:17:34,545 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users JOIN addresses ON users.id = addresses.user_id 
WHERE addresses.email_address = ?
2019-10-26 21:17:34,547 INFO sqlalchemy.engine.base.Engine ('jack@google.com',)


[<User(name='jack', fullname='Jack Bean', nickname='gjffdd')>]

In [93]:
session.query.join(Address, User.id==Address.user_id)

AttributeError: 'function' object has no attribute 'join'

In [94]:
query=session.query(User,Address).select_from(Address).join(User)

In [96]:
from sqlalchemy.orm import aliased
adalias1 = aliased(Address)
adalias2 = aliased(Address)
for username, email1,email2 in session.query(User.name, adalias1.email_address,adalias2.email_address).\
    join(adalias1, User.addresses).\
    join(adalias2, User.addresses).\
    filter(adalias1.email_address=='jack@google.com').\
    filter(adalias2.email_address=='j25@google.com'):
    print(username, email1, email2)

2019-10-26 21:31:06,539 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, addresses_1.email_address AS addresses_1_email_address, addresses_2.email_address AS addresses_2_email_address 
FROM users JOIN addresses AS addresses_1 ON users.id = addresses_1.user_id JOIN addresses AS addresses_2 ON users.id = addresses_2.user_id 
WHERE addresses_1.email_address = ? AND addresses_2.email_address = ?
2019-10-26 21:31:06,540 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 'j25@google.com')
jack jack@google.com j25@google.com


In [97]:
from sqlalchemy.sql import func
stmt = session.query(Address.user_id, func.count('*').\
                    label('address_count')).\
                    group_by(Address.user_id).subquery()